### Callin Switzer

Original: 3 Nov 2016
Update: Feb 10, 2016

Note: Use a custom Python environment [vis_env]

- Read in all the Kalmia polygon images (from R: 2_3_ImportDigitizedPolygonsAndSaveAsImages_Submission.RMD)

- Extrude all of the images so that they are in 3D space

- Resample all of those volumes, and randomly rotate them into 1/10 of 360 degrees

- Add all of the resampled volumes together to get a "mean" location that pollen will be

- Visualize and save a heatmap in 3D space

In [1]:
# open CV
import cv2

# helpers
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk
import os
import pandas as pd
import time
import sys

In [2]:
# scikit image
import skimage
from skimage import io

In [3]:
# image vis packages
from mpl_toolkits.mplot3d import Axes3D
import scipy
import scipy.ndimage

# 3D vis and stats
from mayavi import mlab
from scipy import stats

In [ ]:
# %qtconsole

In [4]:
os.chdir("/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/Manuscript/DatasetsSupplemental/KalmiaDigitizedPolygon")

In [5]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
photoList = [ x for x in onlyfiles if not x.startswith('.') and x.endswith('.png')] # all of the polygons

In [6]:
len(photoList) # 29 polygons

29

In [7]:
# show 2D polygon
img = io.imread(photoList[8], as_grey=True)
io.imshow(img)

# Read in all images, resize, and put in an array

In [8]:
# read in all images in direcory, resize, and make a big numpy array
newSideLen = 200
img = np.array(cv2.imread(photoList[1], 0))
r = float(newSideLen) / img.shape[1]
dim = (int(newSideLen), int(img.shape[0] * r))

stackedImgs = np.ones((newSideLen, newSideLen, len(photoList)))


for hh in range(len(photoList)):
    img = np.array(cv2.imread(photoList[hh], 0)) # read in image black and white

    # perform the resizing
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    # convert to binary
    resized = np.array((resized < 255) * 1.0)

    # clear edges of resized image (grid from graph)
    resized[:, 0:newSideLen/20] = 0
    resized[newSideLen - newSideLen/20:newSideLen, :] = 0
    
    # add to stacked array
    stackedImgs[:,:,hh] = resized

In [9]:
# visualize a single extruded volume
# get image to rotate
mlab.close(all = True)
kk = 8


pollWidth = newSideLen / 100 / 2
midpt = newSideLen / 2

# make temporary array
stackedRot = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
w,h = stackedRot.shape[:2]

resized = stackedImgs[:,:,kk]
resized_flip = np.rot90(resized, 3)

# get rotatation matrix
M = cv2.getRotationMatrix2D((newSideLen/2,newSideLen/2), 36*0, 1.0)

# reset blank 3D array
stackedRot = stackedRot * 0.0

# get image to rotate into 3D space
stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] = stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] + resized_flip

fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0, 0, 0))


mlab.contour3d(stackedRot,colormap = 'Greys')
mlab.orientation_axes()

In [ ]:
# save image, after it's positioned how you want
# saveDir = '/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/Manuscript/Media/'
# mlab.savefig(saveDir + 'SinglePollenVolume' + '.png', size = (1440, 1024))

# Resample images and put in 3D space

In [10]:
def resampRotate(stackedImgs):
    '''
    Resample all the images (same number of photos that I took)
    Randomly rotate images in 1/10 of 360 degree increments
    Put into 3D space
    '''
    
    # prepare to resample resized images
    pollWidth = newSideLen / 100 / 2
    midpt = newSideLen / 2

    # make temporary array
    stackedRot = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
    w,h = stackedRot.shape[:2]


    # make final array
    stackedRot_fin = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
    
    
    # resample all images
    photoSamp = np.random.choice(range(len(photoList)), size = len(photoList), replace = True)
    # resample rotation angles
    angles = np.random.choice(range(10), len(photoList), True)


    for kk in photoSamp:

        # get image to rotate
        resized = stackedImgs[:,:,kk]
        resized_flip = np.rot90(resized, 3)

        # get rotatation matrix
        M = cv2.getRotationMatrix2D((newSideLen/2,newSideLen/2), 36*angles[kk], 1.0)

        # reset blank 3D array
        stackedRot = stackedRot * 0.0

        # extrude image into 3D space
        stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] = stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] + resized_flip

        # rotate image in 3D, by doing one level at a time
        for ii in range(newSideLen):
            stackedRot_fin[:,:,ii] = ((cv2.warpAffine(stackedRot[:,:,ii], M, (w, h)) > 0.3) * 1 +
                                  stackedRot_fin[:,:,ii])
        
    return stackedRot_fin

In [11]:
# generate a single bootstrap sample of a stacked array
rotimg = resampRotate(stackedImgs)

In [12]:
# Visualize single bootstrap sample
mlab.close(all = True)
density = rotimg / (29.0)

fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour3d(density, contours= [0.001], opacity = 0.2, colormap = "hsv")
mlab.orientation_axes()
colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')
colorbar.scalar_bar_representation.position = [0.8, 0.2]
colorbar.scalar_bar_representation.position2 = [0.05, 0.6]

In [ ]:
# repeat the resampling method 500 times

# res_fin = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0

# stt = time.time()
# for ii in range(500):
#     tmp = resampRotate(stackedImgs)
#     res_fin = res_fin + tmp
#     print ii, time.time() - stt # takes about 8 seconds each

In [ ]:
# save numpy array, so I don't have to keep running the simulation
# outfile = '/Users/callinswitzer/Desktop/array5.npy'
# np.save(outfile, res_fin)

# Once simulations are done, start at this point

Visualize the 3D Heatmap and contours

Save figures for publication

In [13]:
# start here, if not conducting the simulations
# load np array
res_fin = np.load('/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/Manuscript/DatasetsSupplemental/array5.npy')
intPoints = np.round(np.array([[768, 1694],[1000, 1694],[1231, 1694]]) / 10.0) # points that correspond to flowers
density = res_fin / (29 * 500) # dividing array by max possible value (29 videos, 500 resamples)

In [14]:
# visualize a few contours from all bootstrap samples

mlab.close(all = True)
density = res_fin / (29 * 500) 

# this cuts the array in half, so we can see inside
# density[101:200, :, :] = 0

fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour3d(density, contours= [0.01, 0.2, 0.4], opacity = 0.2, colormap = "hot")
mlab.orientation_axes()
colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')
colorbar.scalar_bar_representation.position = [0.8, 0.2]
colorbar.scalar_bar_representation.position2 = [0.05, 0.6]
#     mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
#                         scale_mode='none', scale_factor=2, opacity = 1)

# mlab.savefig('halfSizeKalmiaMap' + '.png', size = (1440, 1024))

In [15]:
# visualize array and save figure for paper

mlab.close(all = True)
density = res_fin / (29 * 500) 
# this cuts the array in half
density[101:200, :, :] = 0

fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour3d(density, contours= 50, opacity = 0.2, colormap = "hot")
mlab.orientation_axes()
colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')
colorbar.scalar_bar_representation.position = [0.8, 0.2]
colorbar.scalar_bar_representation.position2 = [0.05, 0.6]
#     mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
#                         scale_mode='none', scale_factor=2, opacity = 1)

# mlab.savefig('halfSizeKalmiaMap' + '.png', size = (1440, 1024))

In [ ]:
# # Visualize with slice cut into array -- make into a movie
# newSideLen = 200
# os.chdir('/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/KalmiaProject/KalmMovie3')

# for ctr in range(40):
    
#     mlab.close(all = True)
#     density = res_fin / (29 * 500)
#     # density[:, 102:newSideLen, (5 * ctr):newSideLen] = 0
#     density[(5 * ctr):newSideLen, 102:newSideLen, :] = 0

#     fig = mlab.figure(size = (1024,768),
#                 bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
#     #mlab.contour3d(density, contours= [0.001, 0.05, 0.2, 0.5], opacity = 0.2, colormap = "jet")
#     mlab.contour3d(density, contours= 50, opacity = 0.2, colormap = "hot")
#     #mlab.axes(extent = [25, 175, 25, 175, 15, 175], y_axis_visibility = False)
#     mlab.orientation_axes()
#     colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')
#     colorbar.scalar_bar_representation.position = [0.8, 0.2]
#     colorbar.scalar_bar_representation.position2 = [0.05, 0.6]
# #     mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
# #                         scale_mode='none', scale_factor=2, opacity = 1)

#     mlab.savefig(str(ctr + 41).zfill(3) + '.png', size = (1024, 1024))
#     print ctr

In [ ]:
# make contour plot
mlab.close(all = True)
fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
aa = mlab.contour_surf(density[100, :,:], contours = [0.009,0.02, 0.05, 0.11, 0.4], colormap = 'hot')
colorbar = mlab.colorbar(orientation = 'vertical', label_fmt = '%.1f')

# add reference points to show where flower would be positioned
mlab.points3d( 100 - intPoints[:,0], 99 - intPoints[:,1] + 1, [0, 0, 0], colormap = 'hot', 
                    scale_mode='none', scale_factor=2, opacity = 1)

In [ ]:
# mlab.savefig('contourMap' + '.png', size = (1440, 1024))

In [ ]:
# show installed packages and versions
!pip freeze 

In [ ]:
# print system info
import IPython
print IPython.sys_info()